In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('NLP').getOrCreate()

In [5]:
df = spark.read.csv('smsspamcollection/SMSSpamCollection', inferSchema = True, sep = '\t')

In [6]:
df.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [7]:
df = df.withColumnRenamed('_c0', 'class').withColumnRenamed('_c1', 'text')

In [8]:
df.show(5)

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
+-----+--------------------+
only showing top 5 rows



In [9]:
from pyspark.sql.functions import length

In [11]:
df = df.withColumn('length', length(df['text']))

In [12]:
df.show(5)

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
+-----+--------------------+------+
only showing top 5 rows



In [13]:
df.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



### **Data Preprocessing**

In [14]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer

In [15]:
tokenizer = Tokenizer(inputCol = 'text', outputCol='token_text')

In [17]:
tokenizer.transform(df).show(5)

+-----+--------------------+------+--------------------+
|class|                text|length|          token_text|
+-----+--------------------+------+--------------------+
|  ham|Go until jurong p...|   111|[go, until, juron...|
|  ham|Ok lar... Joking ...|    29|[ok, lar..., joki...|
| spam|Free entry in 2 a...|   155|[free, entry, in,...|
|  ham|U dun say so earl...|    49|[u, dun, say, so,...|
|  ham|Nah I don't think...|    61|[nah, i, don't, t...|
+-----+--------------------+------+--------------------+
only showing top 5 rows



In [28]:
stop_remove = StopWordsRemover(inputCol='token_text', outputCol='stop_token')
count_vec = CountVectorizer(inputCol = 'stop_token', outputCol = 'c_vec')

In [29]:
idf = IDF(inputCol = 'c_vec', outputCol = 'tf_idf')

In [30]:
ham_spam_to_numeric = StringIndexer(inputCol = 'class', outputCol = 'label')

In [22]:
from pyspark.ml.feature import VectorAssembler

In [23]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'], outputCol='features')

In [24]:
from pyspark.ml.classification import NaiveBayes

In [38]:
nb = NaiveBayes()

In [25]:
from pyspark.ml import Pipeline

In [31]:
data_prep_pipe = Pipeline(stages = [ham_spam_to_numeric, tokenizer, stop_remove,
                                   count_vec, idf, clean_up])

In [32]:
cleaner = data_prep_pipe.fit(df)

In [33]:
clean_data = cleaner.transform(df)

In [34]:
clean_data.columns

['class',
 'text',
 'length',
 'label',
 'token_text',
 'stop_token',
 'c_vec',
 'tf_idf',
 'features']

In [35]:
clean_data = clean_data.select('label', 'features')

In [36]:
clean_data.show(5)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,301,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
+-----+--------------------+
only showing top 5 rows



In [37]:
train_data, test_data = clean_data.randomSplit([0.7, 0.3])

### **Model Building**

In [39]:
spam_detector = nb.fit(train_data)

In [40]:
df.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [41]:
test_results = spam_detector.transform(test_data)

In [42]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,41,...|[-1060.4062133367...|[1.0,3.3199371641...|       0.0|
|  0.0|(13424,[0,1,3,9,1...|[-572.94237611087...|[1.0,2.5459964297...|       0.0|
|  0.0|(13424,[0,1,4,50,...|[-845.25310539219...|[1.0,2.2783870174...|       0.0|
|  0.0|(13424,[0,1,7,8,1...|[-872.37664310114...|[1.0,6.2222391651...|       0.0|
|  0.0|(13424,[0,1,9,14,...|[-539.55947440068...|[1.0,4.2197257827...|       0.0|
|  0.0|(13424,[0,1,12,33...|[-448.79277503567...|[1.0,1.5100150694...|       0.0|
|  0.0|(13424,[0,1,20,27...|[-969.17149694408...|[0.99999999999999...|       0.0|
|  0.0|(13424,[0,1,21,27...|[-992.39400204681...|[1.0,6.8360906366...|       0.0|
|  0.0|(13424,[0,1,24,31...|[-356.09232928705...|[1.0,2.8308221239...|       0.0|
|  0.0|(13424,[0

In [43]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [44]:
acc_eval = MulticlassClassificationEvaluator()

In [45]:
acc = acc_eval.evaluate(test_results)

In [46]:
print(f"Accuracy of NB model: {acc}")

Accuracy of NB model: 0.9233062819717635
